![Colegio Bourbaki](./Images/Bourbaki.png)

## Procesamiento de Lenguaje Natural

### Contexto

El objetivo de este notebook es hacer una demostración de la creación de chatbots estilo ChatGPT con conocimiento de datos específicos.

En primer lugar, enseñaremos cómo conectar con el API de Hugging Face para utilizar modelos guardados en ese Hub desde código.

Después, veremos cómo podemos introducir material a la base de conocimiento del chatbot, para así obtener respuestas más personalizadas.

### Librerías

In [1]:
# NLP / LLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    BitsAndBytesConfig
)

# LangChain
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

# Utils
import os
from pdfminer.high_level import extract_text
import torch

Primero, definimos el modelo

In [4]:
# Modelo open-weights
model_name = "microsoft/Phi-3.5-mini-instruct"

In [5]:
# Configuración 4-bit para que corra localmente
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
# Pipeline de generación "crudo"
generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=64,
    do_sample=True,
    temperature=0.2,
)

Device set to use cuda:0


In [9]:
prompt = "Explicame en pocas palabras qué es el fine-tuning con LoRA."

In [12]:
outputs = generation_pipeline(prompt)
print(outputs[0]["generated_text"])

Explicame en pocas palabras qué es el fine-tuning con LoRA.

El fine-tuning con LoRA (Low-Rank Adaptation) es una técnica utilizada en el aprendizaje automático para ajustar modelos pre-entrenados, especialmente en el contexto del procesamiento de lenguaje natural (NLP) y visión por computadora. LoRA permite ajustar finamente un modelo sin modificar significativally sus pesos originales, lo que puede ser beneficioso para conservar el conocimiento que el modelo ya ha aprendido y para reducir el costo computacional y el tiempo de


In [14]:
preguntas = [
    "¿Qué es un modelo de lenguaje grande (LLM)? Respondé en menos de 3 líneas.",
    "¿Qué es RAG (Retrieval-Augmented Generation)? Explicalo con un ejemplo.",
    "¿En qué se diferencia fine-tuning clásico de LoRA?",
]

for i, q in enumerate(preguntas, start=1):
    print(f"\n--- Pregunta {i} ---")
    outputs = generation_pipeline(q)
    print(outputs[0]["generated_text"])


--- Pregunta 1 ---
¿Qué es un modelo de lenguaje grande (LLM)? Respondé en menos de 3 líneas. Un modelo de lenguaje grande (LLM) es un sistema avanzado de inteligencia artificial que ha sido entrenado en vastas cantidades de datos textuales. Utiliza algoritmos de aprendizaje automático para comprender y generar lenguaje humano,

--- Pregunta 2 ---
¿Qué es RAG (Retrieval-Augmented Generation)? Explicalo con un ejemplo.

# Answer
RAG (Retrieval-Augmented Generation) es una técnica en el campo de la inteligencia artificial y el procesamiento de lenguaje natural que combina la generación de texto con la recuperación de información de una base de datos para producir res

--- Pregunta 3 ---
¿En qué se diferencia fine-tuning clásico de LoRA?

Fine-tuning clásico y LoRA (Low-Rank Adaptation) son técnicas utilizadas para adaptar modelos pre-entrenados a nuevas tareas, pero difieren en su enfoque y eficiencia.

1. Fine


Podemos lograr mejores respuestas del modelo si modificamos el atributo **system**.

In [15]:
def build_chat_prompt(system_msg, user_msg):
    return f"<|system|>\n{system_msg}\n<|user|>\n{user_msg}\n<|assistant|>\n"

In [16]:
sistema = "Eres un asistente de poetas, habilidoso en explicar conceptos complejos de programación creativamente."
usuario = "Compón un poema que explique el concepto de recursión en programación."

In [17]:
prompt = build_chat_prompt(sistema, usuario)

In [19]:
from pprint import pprint

In [21]:
pprint(prompt)

('<|system|>\n'
 'Eres un asistente de poetas, habilidoso en explicar conceptos complejos de '
 'programación creativamente.\n'
 '<|user|>\n'
 'Compón un poema que explique el concepto de recursión en programación.\n'
 '<|assistant|>\n')


In [ ]:
from pprint import pprint

In [ ]:
pprint(prompt)

In [22]:
out = generation_pipeline(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
)

print(out[0]["generated_text"])

<|system|>
Eres un asistente de poetas, habilidoso en explicar conceptos complejos de programación creativamente.
<|user|>
Compón un poema que explique el concepto de recursión en programación.
<|assistant|>
 En el reino de la codificación, donde los pensamientos se concretan,
Existe un concepto, sutil, pero significativo.
Es una estrategia, un enfoque, recurso en recurso,
Una técnica que los creadores de programas usan con gusto.

Una recursión, como un viaje sin fin de la noche,
Donde cada paso conduce a otro en el mismo camino.
Un enfoque de pensamiento, repetitivo, pero elegante,
Donde cada llamada llama a la siguiente, una verdad inexorable.

Una función, se llama a sí misma, una llamada dentro de una llamada,
Una búsqueda infinita, que se ve desde fuera.
Sin embargo, no es una marcha interminable,
Porque cada llamada tiene un fin, una meta.

En cada paso, una pieza del rompecabezas,
Un fragmento de la solución, que se acerca.
Sin necesidad de salir, sin recorrido extra


In [ ]:
pprint(out)

### Creación de un asistente especializado

A su vez, podemos aprovechar aún más las capacidades de loss LLM haciendo una especie de fine-tuning. La idea consiste en alimentar al modelos con documentos propios para así lograr respuestas informadas sobre ellos.

Esto es posible a través de los encajes y la generación de una base de datos vectorizada.

Primero, extraemos texto desde archivos pdfs...


... y extramos el texto de un pdf para después guardarlo en .txt.

In [23]:
pdf_path = "/media/pdconte/hdd/Pablo/Personal/Colegio_Bourbaki/Natural_Language_Processsing/Semana5/Data/Feynman1982_Article_SimulatingPhysicsWithComputers.pdf"
output_txt_path = "./Data/output.txt"

# Extraer texto del PDF a un archivo de texto
text = extract_text(pdf_path)

os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)
with open(output_txt_path, "w", encoding="utf-8") as f:
    f.write(text)

print("Texto extraído y guardado en", output_txt_path)

Texto extraído y guardado en ./Data/output.txt


In [28]:
text

'International Journal of Theoretical Physics, VoL 21, Nos. 6/7,  1982 \n\nSimulating Physics with Computers \n\nRichard P. Feynman \n\nDepartment of Physics, California Institute of Technology, Pasadena, California 91107 \n\nReceived May 7, 1981 \n\n1.  I N T R O D U C T I O N  \n\nOn  the  program  it  says  this  is  a  keynote  speech--and  I  don\'t  know \nwhat  a  keynote speech is.  I  do not intend  in  any way to suggest what  should \nbe  in  this  meeting  as  a  keynote of  the  subjects  or  anything  like  that.  I  have \nmy  own  things  to  say  and  to  talk  about  and  there\'s  no  implication  that \nanybody  needs  to  talk  about  the  same  thing  or  anything  like  it.  So  what  I \nwant  to  talk  about  is  what  Mike  Dertouzos  suggested  that  nobody  would \ntalk  about.  I  want  to  talk  about  the  problem  of  simulating  physics  with \ncomputers  and  I  mean  that  in  a  specific way  which  I  am  going  to  explain. \nThe  reason  for  doin

In [26]:
loader = TextLoader("./Data/output.txt")
documents = loader.load()

text_splitter = CharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100, separator="\n"
)

texts = text_splitter.split_documents(documents)

In [27]:
texts[0]

Document(metadata={'source': './Data/output.txt'}, page_content="International Journal of Theoretical Physics, VoL 21, Nos. 6/7,  1982 \nSimulating Physics with Computers \nRichard P. Feynman \nDepartment of Physics, California Institute of Technology, Pasadena, California 91107 \nReceived May 7, 1981 \n1.  I N T R O D U C T I O N  \nOn  the  program  it  says  this  is  a  keynote  speech--and  I  don't  know \nwhat  a  keynote speech is.  I  do not intend  in  any way to suggest what  should \nbe  in  this  meeting  as  a  keynote of  the  subjects  or  anything  like  that.  I  have \nmy  own  things  to  say  and  to  talk  about  and  there's  no  implication  that \nanybody  needs  to  talk  about  the  same  thing  or  anything  like  it.  So  what  I \nwant  to  talk  about  is  what  Mike  Dertouzos  suggested  that  nobody  would \ntalk  about.  I  want  to  talk  about  the  problem  of  simulating  physics  with \ncomputers  and  I  mean  that  in  a  specific way  which  I

In [29]:
len(texts)

72

En seguida, cargamos el documento a la base de datos.

In [30]:
# Embeddings HuggingFace (sin costo de API)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Crear / persistir base vectorial con Chroma
db = Chroma.from_documents(texts, embeddings, collection_name="docs_rag")

Este modelo ahora se puede utilizar como un ChatGPT con conocimiento especializado:

In [33]:
torch.cuda.empty_cache()

In [34]:
generation_pipeline_db = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.2,
    return_full_text=False,
)

local_llm = HuggingFacePipeline(pipeline=generation_pipeline_db)

Device set to use cuda:0


In [35]:
retriever = db.as_retriever(search_kwargs={"k": 1})
qa = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type="stuff",
    retriever=retriever,
)

In [38]:
# Ejemplo de consulta
query = "¿De qué trata el documento? Resume los puntos principales."
result = qa.invoke({"query": query})

In [39]:
respuesta = result["result"]
respuesta

' El documento trata sobre una conferencia o reunión donde Richard P. Feynman pronuncia un discurso de toma de posesión o keynote. En su discurso, Feynman aborda el tema de simular la física utilizando computadoras. Aunque no proporciona detalles específicos sobre cómo se aborda este tema, indica que su enfoque es diferente de lo que Mike Dertouzos sugiere que deberían hablar. Los puntos principales son la naturaleza del discurso de Feynman'

In [40]:
query = "How photons are polarized?"
result = qa.invoke({"query": query})
result['result']

' Photons are polarized into one of four possible beams, determined by the detectors they pass through. The polarization is such that a photon will either go into the ordinary ray or the extraordinary one, but not both. The probability of a photon being in either beam is always 100%, but it will never be found in both.\n\n'

In [41]:
query = "Expand the concept of negative probabilities"
result = qa.invoke({"query": query})
result["result"]

"\n\nNegative probabilities are not a concept that exists in the standard framework of probability theory. Probabilities are always non-negative and range between 0 and 1 inclusive. This is because probability is a measure of certainty or likelihood of an event occurring, and it doesn't make sense to have a negative measure of certainty.\n\nHowever, in certain advanced fields like quantum mechanics, the concept of negative probabilities can appear in the form of complex numbers, but these are not negative probabilities in the traditional sense. In quantum mechanics, the square of the amplitude of a state"

In [42]:
query = "What is a quantum computer?"
result = qa.invoke({"query": query})
result["result"]

' A quantum computer is a type of computing device that uses the principles of quantum mechanics to process information. Unlike classical computers, which use bits as the smallest unit of data (which can be either 0 or 1), quantum computers use quantum bits, or qubits, which can exist in multiple states simultaneously due to the principle of superposition. This allows quantum computers to potentially solve complex problems much faster than classical computers. However, the field of quantum computing is still largely theoretical and experimental, and the concept of a quantum computer as it relates to the world view of quantum mechanics is not fully understood even by experts.'

Podemos comparar la salida con el modelo original:

In [43]:
out = generation_pipeline(
    query,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
)

print(out[0]["generated_text"])

What is a quantum computer? A quantum computer uses quantum bits, or qubits, which can be in a state of 0 and 1 simultaneously, due to the superposition principle. This allows a quantum computer to process a vast amount of data simultaneously, giving it the potential to solve complex problems much faster than classical computers.

2. What is the difference between classical and quantum computing? Classical computing uses bits as the smallest unit of data, which can be either 0 or 1. Quantum computing uses quantum bits or qubits, which can be both 0 and 1 at the same time due to the superposition principle. This gives


In [44]:
query = "¿De qué trata el documento? Resume los puntos principales."

In [45]:
out = generation_pipeline(
    query,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
)

print(out[0]["generated_text"])

¿De qué trata el documento? Resume los puntos principales.

# Answer
Lamentablemente, sin el texto o el contenido del documento, no puedo identificar ni resumir los puntos principales. Por favor, proporcías el documento o sus puntos principales.


### Preguntas

**Uso de LLM:**

4) ¿Qué ventajas ofrece usar un LLM para la creación de chatbots comparado con versiones anteriores?
5) ¿Cómo se formulan las peticiones al modelo para generar respuestas coherentes y relevantes?
6) ¿Qué limitaciones tienes estos modelos y cómo puedes mitigarlas?

**Introducción de Material a la Base de Conocimiento:**

7) ¿Cómo puedes personalizar las respuestas del modelo utilizando información específica?
8) ¿Cuál es la importancia de la relevancia y precisión del material que se introduce en la base de conocimientos del bot?
9) ¿Qué estrategias se pueden utilizar para mantener actualizada la base de conocimientos del chatbot?

**Personalización y Respuestas del Chatbot:**

10) ¿De qué manera se puede ajustar el tono o el estilo de las respuestas que genera el modelo?
11) ¿Cómo afecta el contexto proporcionado a las respuestas generadas por el chatbot?
    Describe un método para evaluar la precisión y utilidad de las respuestas del chatbot.

**Problemas Éticos y de Privacidad:**

12) ¿Cuáles son las consideraciones éticas al utilizar modelos de lenguaje generativos en un chatbot?
13) ¿Cómo debería manejar un chatbot las solicitudes de datos personales o sensibles de los usuarios?
14) ¿Qué medidas se pueden tomar para garantizar la privacidad y la seguridad de los usuarios al interactuar con un chatbot?

![Lenguaje Matemático](./Images/Matematicas.png)